## Case for shewanella
> 2023-01-29
### 1.import packages

In [2]:
import sys,os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
sys.path.append("../tools/")
import tools.funclib as funclib
import numpy as np
import pandas as pd
from keras.models import load_model
from tools.Attention import Attention

import config as cfg
import tools.embedding_esm as ebdesm
from pandarallel import pandarallel 
pandarallel.initialize() 

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 52 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## 2. Read shewanella fasta file

In [2]:
data_shewanella = funclib.load_fasta_to_table(cfg.FILE_CASE_SHEWANELLA_FASTA)
data_shewanella.head(3)

,id,seq
0,WP_005054087.1,MTQPFNFEQALKDLQSGKSLTGKDSILGPLIKQLTEAALQAELEQH...
1,WP_006079870.1,MSRVCQVTGKKPMVGNNRSHAKNATRRRFLPNLQNHRFWLEEEKRF...
2,WP_006079871.1,MAKAKGNREKIKLVSTAKTGHFYTTEKNKRNMPEKMEIKKFDPVIR...


## 3. Embedding

In [6]:
# final_rep0, final_rep32, final_rep33 = ebdesm.get_rep_multi_sequence(data_shewanella)
# final_rep32.to_feather(cfg.RESULTSDIR+'ebdesm_32_shewanella.feather')


## 4.读取测试数据

In [2]:
# embeddin 方法
EMBD_METHO='esm32'
EMBD_WH = 1280
TOPK=20

ecs_2 = [True, False]
ecs_8 = [1,2,3,4,5,6,7,8]
label_model_ec = pd.read_feather(f'{cfg.MODELDIR}/task3_labels.feather').label_multi.to_list()

ebdesm_32_shewanella = pd.read_feather(cfg.RESULTSDIR+'ebdesm_32_shewanella.feather')
test_feature = ebdesm_32_shewanella.iloc[:, 1:]
test_feature =np.reshape(np.array(test_feature),(test_feature.shape[0],1,EMBD_WH))
test_res = pd.DataFrame(ebdesm_32_shewanella.id.copy().astype('str'))



In [3]:
pd.read_feather(f'{cfg.MODELDIR}/task3_labels.feather').label_multi.to_list()

['3.5.1.108',
 '2.3.1.156',
 '4.2.3.93',
 '2.4.1.175',
 '5.3.1.4',
 '2.4.99.3',
 '1.14.11.29',
 '2.4.1.83',
 '1.3.99.36',
 '4.2.1.68',
 '1.1.1.31',
 '2.4.1.10',
 '4.2.3.25',
 '1.12.1.2',
 '1.3.1.1',
 '1.14.13.11',
 '6.3.2.5',
 '3.2.1.165',
 '1.2.1.87',
 '2.1.1.210',
 '3.2.1.180',
 '2.7.7.72',
 '2.7.7.46',
 '4.4.1.24',
 '3.5.3.18',
 '1.14.11.55',
 '3.4.24.87',
 '2.3.1.57',
 '2.5.1.99',
 '3.1.1.17',
 '1.14.19.6',
 '1.14.15.6',
 '4.99.1.3',
 '3.2.1.48',
 '2.1.1.219',
 '1.1.1.381',
 '1.17.4.4',
 '3.5.1.105',
 '4.1.2.53',
 '1.21.4.1',
 '5.1.3.20',
 '1.1.1.309',
 '1.14.13.197',
 '1.5.1.38',
 '6.5.1.3',
 '1.14.18.1',
 '1.4.5.-',
 '2.7.-.-',
 '1.7.1.3',
 '1.97.1.12',
 '4.2.1.162',
 '2.4.2.37',
 '1.3.7.1',
 '3.6.3.39',
 '3.4.23.29',
 '2.1.1.141',
 '3.1.1.59',
 '1.14.21.2',
 '2.3.1.110',
 '1.14.11.13',
 '4.2.3.30',
 '2.1.1.179',
 '5.1.3.31',
 '2.4.1.141',
 '2.1.1.151',
 '3.4.21.36',
 '3.1.6.13',
 '6.3.2.47',
 '3.4.22.33',
 '1.21.99.1',
 '2.4.1.52',
 '2.6.1.14',
 '1.3.1.29',
 '1.1.1.219',
 '1.13.

## 5. DMLF [task1, task2, task3]

In [3]:
# task1
model_t1 = load_model(f'{cfg.MODELDIR}/task1_2022.h5', custom_objects={"Attention": Attention},compile=False)
predicted = model_t1.predict(test_feature)
test_res['dmlf_isenzyme'] = np.argmax(predicted, axis=-1).astype('bool')

# Task2
list_func = list()
model_t2 = load_model(f'{cfg.MODELDIR}/task2_model_2022_label8.h5',custom_objects={"Attention": Attention},compile=False)
predicted = model_t2.predict(test_feature)
for i in range(predicted.shape[0]):
    list_func.append(sorted(dict(zip(set(ecs_8),predicted[i])).items(),key = lambda x:x[1], reverse = True)[0][0])
test_res['dmlf_functions']=list_func


#Task3
model_t3 = load_model(f'{cfg.MODELDIR}/task3_esm32_2022.h5',custom_objects={"Attention": Attention},compile=False)
predicted = model_t3.predict(test_feature)
test_res['sorted_ec'] = pd.DataFrame(predicted).parallel_apply(lambda x: sorted(dict(zip((label_model_ec), x.to_list())).items(),key = lambda x:x[1], reverse = True), axis=1)
test_res['ec_dmlf']=test_res.parallel_apply(lambda x: (list(dict(x.sorted_ec[0:x.dmlf_functions]).keys())), axis=1)
test_res['ec_dmlf']=test_res.parallel_apply(lambda x: '-' if x.dmlf_isenzyme ==False else ','.join(x.ec_dmlf), axis=1)
test_res['ec_dmlf_topk']=test_res.parallel_apply(lambda x: x.sorted_ec[0:TOPK], axis=1)

test_res

132/132 [==============================] - 1s 1ms/step


,id,dmlf_isenzyme,dmlf_functions,sorted_ec,ec_dmlf,ec_dmlf_topk
0,WP_005054087.1,False,1,"[(3.1.-.-, 0.1445898711681366), (3.1.22.-, 0.0...",-,"[(3.1.-.-, 0.1445898711681366), (3.1.22.-, 0.0..."
1,WP_006079870.1,False,1,"[(3.1.26.5, 0.11312833428382874), (3.1.1.29, 0...",-,"[(3.1.26.5, 0.11312833428382874), (3.1.1.29, 0..."
2,WP_006079871.1,False,1,"[(2.7.7.6, 0.6336387395858765), (3.1.26.5, 0.0...",-,"[(2.7.7.6, 0.6336387395858765), (3.1.26.5, 0.0..."
3,WP_006080725.1,False,1,"[(3.1.26.5, 0.10884372144937515), (2.7.7.6, 0....",-,"[(3.1.26.5, 0.10884372144937515), (2.7.7.6, 0...."
4,WP_006080791.1,False,1,"[(3.1.-.-, 0.9882077574729919), (4.2.99.18, 0....",-,"[(3.1.-.-, 0.9882077574729919), (4.2.99.18, 0...."
...,...,...,...,...,...,...
4197,WP_238560620.1,False,1,"[(3.1.-.-, 0.5166441202163696), (5.99.1.2, 0.0...",-,"[(3.1.-.-, 0.5166441202163696), (5.99.1.2, 0.0..."
4198,WP_238560621.1,False,1,"[(3.1.-.-, 0.1078561320900917), (2.3.1.48, 0.0...",-,"[(3.1.-.-, 0.1078561320900917), (2.3.1.48, 0.0..."
4199,WP_238560622.1,False,1,"[(3.1.-.-, 0.227324977517128), (3.1.21.-, 0.00...",-,"[(3.1.-.-, 0.227324977517128), (3.1.21.-, 0.00..."
4200,WP_238560623.1,False,1,"[(3.1.-.-, 0.9987987279891968), (3.4.24.-, 0.0...",-,"[(3.1.-.-, 0.9987987279891968), (3.4.24.-, 0.0..."


## 6.Balst

In [4]:
sprot = pd.read_feather(cfg.DATADIR+'uniprot/snap201802.feather')
data_shewanella = funclib.load_fasta_to_table(cfg.FILE_CASE_SHEWANELLA_FASTA)

res_diamond = funclib.getblast(train=sprot[['id','seq']], test=data_shewanella[['id','seq']])
res_diamond = res_diamond [['id','sseqid','pident']].merge(sprot[['id','ec_number']], how='left', left_on='sseqid', right_on='id')
res_diamond = res_diamond[['id_x', 'ec_number','pident']].rename(columns={'id_x':'id','ec_number':'ec_blast', 'pident':'pident_blast'})

test_res = test_res.merge(res_diamond, on='id', how='left')
test_res = test_res.reset_index(drop=True)
test_res.ec_blast = test_res.ec_blast.fillna('-')
test_res

Write finished
Write finished
diamond makedb --in /tmp/train.fasta -d /tmp/train.dmnd --quiet
diamond blastp -d /tmp/train.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1 --quiet


,id,dmlf_isenzyme,dmlf_functions,sorted_ec,ec_dmlf,ec_dmlf_topk,ec_blast,pident_blast
0,WP_005054087.1,False,1,"[(3.1.-.-, 0.1445898711681366), (3.1.22.-, 0.0...",-,"[(3.1.-.-, 0.1445898711681366), (3.1.22.-, 0.0...",-,40.9
1,WP_006079870.1,False,1,"[(3.1.26.5, 0.11312833428382874), (3.1.1.29, 0...",-,"[(3.1.26.5, 0.11312833428382874), (3.1.1.29, 0...",-,100.0
2,WP_006079871.1,False,1,"[(2.7.7.6, 0.6336387395858765), (3.1.26.5, 0.0...",-,"[(2.7.7.6, 0.6336387395858765), (3.1.26.5, 0.0...",-,100.0
3,WP_006080725.1,False,1,"[(3.1.26.5, 0.10884372144937515), (2.7.7.6, 0....",-,"[(3.1.26.5, 0.10884372144937515), (2.7.7.6, 0....",-,100.0
4,WP_006080791.1,False,1,"[(3.1.-.-, 0.9882077574729919), (4.2.99.18, 0....",-,"[(3.1.-.-, 0.9882077574729919), (4.2.99.18, 0....",-,100.0
...,...,...,...,...,...,...,...,...
4197,WP_238560620.1,False,1,"[(3.1.-.-, 0.5166441202163696), (5.99.1.2, 0.0...",-,"[(3.1.-.-, 0.5166441202163696), (5.99.1.2, 0.0...",-,NaN
4198,WP_238560621.1,False,1,"[(3.1.-.-, 0.1078561320900917), (2.3.1.48, 0.0...",-,"[(3.1.-.-, 0.1078561320900917), (2.3.1.48, 0.0...",-,NaN
4199,WP_238560622.1,False,1,"[(3.1.-.-, 0.227324977517128), (3.1.21.-, 0.00...",-,"[(3.1.-.-, 0.227324977517128), (3.1.21.-, 0.00...",-,NaN
4200,WP_238560623.1,False,1,"[(3.1.-.-, 0.9987987279891968), (3.4.24.-, 0.0...",-,"[(3.1.-.-, 0.9987987279891968), (3.4.24.-, 0.0...",-,NaN


### 7. Integrate Output

In [5]:
def ec_compare(ec_str1, ec_str2, sep=','):
    ec1= ec_str1.split(sep)
    ec2= ec_str2.split(sep)
    if set(ec1) == set(ec2):
        return True
    else:
        return False

test_res['ec_dmlf']=test_res.parallel_apply(lambda x: x.ec_blast if x.ec_dmlf=='-' else x.ec_dmlf, axis=1) #将序列比对到有EC号，DMLF预测成非酶的，进行EC填充
test_res=test_res[['id', 'ec_dmlf', 'ec_dmlf_topk', 'ec_blast','pident_blast']]
test_res

,id,ec_dmlf,ec_dmlf_topk,ec_blast,pident_blast
0,WP_005054087.1,-,"[(3.1.-.-, 0.1445898711681366), (3.1.22.-, 0.0...",-,40.9
1,WP_006079870.1,-,"[(3.1.26.5, 0.11312833428382874), (3.1.1.29, 0...",-,100.0
2,WP_006079871.1,-,"[(2.7.7.6, 0.6336387395858765), (3.1.26.5, 0.0...",-,100.0
3,WP_006080725.1,-,"[(3.1.26.5, 0.10884372144937515), (2.7.7.6, 0....",-,100.0
4,WP_006080791.1,-,"[(3.1.-.-, 0.9882077574729919), (4.2.99.18, 0....",-,100.0
...,...,...,...,...,...
4197,WP_238560620.1,-,"[(3.1.-.-, 0.5166441202163696), (5.99.1.2, 0.0...",-,NaN
4198,WP_238560621.1,-,"[(3.1.-.-, 0.1078561320900917), (2.3.1.48, 0.0...",-,NaN
4199,WP_238560622.1,-,"[(3.1.-.-, 0.227324977517128), (3.1.21.-, 0.00...",-,NaN
4200,WP_238560623.1,-,"[(3.1.-.-, 0.9987987279891968), (3.4.24.-, 0.0...",-,NaN


## 8. DeepEC

In [6]:
# ! /home/shizhenkun/miniconda3/envs/deepec/bin/python ./baselines/deepec/deepec.py -i ./data/shewanella.faa -o /home/shizhenkun/codebase/DMLF/results/case_shewanella/deepec/

res_deepec= funclib.load_deepec_resluts(f'{cfg.RESULTSDIR}case_shewanella/deepec/DeepEC_Result.txt')
test_res= test_res.merge(res_deepec, on='id', how='left')
test_res.ec_deepec = test_res.ec_deepec.fillna('-')
test_res

,id,ec_dmlf,ec_dmlf_topk,ec_blast,pident_blast,ec_deepec
0,WP_005054087.1,-,"[(3.1.-.-, 0.1445898711681366), (3.1.22.-, 0.0...",-,40.9,-
1,WP_006079870.1,-,"[(3.1.26.5, 0.11312833428382874), (3.1.1.29, 0...",-,100.0,3.1.26.5
2,WP_006079871.1,-,"[(2.7.7.6, 0.6336387395858765), (3.1.26.5, 0.0...",-,100.0,-
3,WP_006080725.1,-,"[(3.1.26.5, 0.10884372144937515), (2.7.7.6, 0....",-,100.0,-
4,WP_006080791.1,-,"[(3.1.-.-, 0.9882077574729919), (4.2.99.18, 0....",-,100.0,-
...,...,...,...,...,...,...
4197,WP_238560620.1,-,"[(3.1.-.-, 0.5166441202163696), (5.99.1.2, 0.0...",-,NaN,-
4198,WP_238560621.1,-,"[(3.1.-.-, 0.1078561320900917), (2.3.1.48, 0.0...",-,NaN,-
4199,WP_238560622.1,-,"[(3.1.-.-, 0.227324977517128), (3.1.21.-, 0.00...",-,NaN,-
4200,WP_238560623.1,-,"[(3.1.-.-, 0.9987987279891968), (3.4.24.-, 0.0...",-,NaN,-


# 9. ECPred

In [7]:
# ! java -jar /home/shizhenkun/codebase/DMLF/baselines/ECPred/ECPred.jar spmap /home/shizhenkun/codebase/DMLF/data/shewanella.faa /home/shizhenkun/codebase/DMLF/baselines/ECPred/ /home/shizhenkun/codebase/DMLF/results/case_gubang/ecpred/ /home/shizhenkun/codebase/DMLF/results/case_shewanella/ecpred/ecpred.tsv
res_ecpred =funclib.load_ecpred_res(f'{cfg.RESULTSDIR}/case_shewanella/ecpred/ecpred.tsv')
test_res= test_res.merge(res_ecpred, on='id', how='left')
test_res.ec_ecpred = test_res.ec_ecpred.fillna('-')
test_res

,id,ec_dmlf,ec_dmlf_topk,ec_blast,pident_blast,ec_deepec,ec_ecpred,pident_ecpred
0,WP_005054087.1,-,"[(3.1.-.-, 0.1445898711681366), (3.1.22.-, 0.0...",-,40.9,-,-,NaN
1,WP_006079870.1,-,"[(3.1.26.5, 0.11312833428382874), (3.1.1.29, 0...",-,100.0,3.1.26.5,-,NaN
2,WP_006079871.1,-,"[(2.7.7.6, 0.6336387395858765), (3.1.26.5, 0.0...",-,100.0,-,-,NaN
3,WP_006080725.1,-,"[(3.1.26.5, 0.10884372144937515), (2.7.7.6, 0....",-,100.0,-,-,NaN
4,WP_006080791.1,-,"[(3.1.-.-, 0.9882077574729919), (4.2.99.18, 0....",-,100.0,-,-,NaN
...,...,...,...,...,...,...,...,...
4197,WP_238560620.1,-,"[(3.1.-.-, 0.5166441202163696), (5.99.1.2, 0.0...",-,NaN,-,-,NaN
4198,WP_238560621.1,-,"[(3.1.-.-, 0.1078561320900917), (2.3.1.48, 0.0...",-,NaN,-,-,NaN
4199,WP_238560622.1,-,"[(3.1.-.-, 0.227324977517128), (3.1.21.-, 0.00...",-,NaN,-,-,NaN
4200,WP_238560623.1,-,"[(3.1.-.-, 0.9987987279891968), (3.4.24.-, 0.0...",-,NaN,-,-,NaN


## 10. CatFAM

In [8]:
# ! cd /home/shizhenkun/download/blast-2.2.26.bin/
# ! /home/shizhenkun/codebase/DMLF/baselines/catfam/source/catsearch.pl -d /home/shizhenkun/codebase/DMLF/baselines/catfam/CatFamDB/CatFam_v2.0/CatFam4D99R -i /home/shizhenkun/codebase/DMLF/data/shewanella.faa -o /home/shizhenkun/codebase/DMLF/results/case_shewanella/catfam/catfam.txt

res_catfam = funclib.load_catfam_res(f'{cfg.RESULTSDIR}/case_shewanella/catfam/catfam.txt')
test_res= test_res.merge(res_catfam, on='id', how='left')
test_res.ec_catfam = test_res.ec_catfam.fillna('-')
test_res

,id,ec_dmlf,ec_dmlf_topk,ec_blast,pident_blast,ec_deepec,ec_ecpred,pident_ecpred,ec_catfam
0,WP_005054087.1,-,"[(3.1.-.-, 0.1445898711681366), (3.1.22.-, 0.0...",-,40.9,-,-,NaN,-
1,WP_006079870.1,-,"[(3.1.26.5, 0.11312833428382874), (3.1.1.29, 0...",-,100.0,3.1.26.5,-,NaN,-
2,WP_006079871.1,-,"[(2.7.7.6, 0.6336387395858765), (3.1.26.5, 0.0...",-,100.0,-,-,NaN,-
3,WP_006080725.1,-,"[(3.1.26.5, 0.10884372144937515), (2.7.7.6, 0....",-,100.0,-,-,NaN,-
4,WP_006080791.1,-,"[(3.1.-.-, 0.9882077574729919), (4.2.99.18, 0....",-,100.0,-,-,NaN,-
...,...,...,...,...,...,...,...,...,...
4227,WP_238560620.1,-,"[(3.1.-.-, 0.5166441202163696), (5.99.1.2, 0.0...",-,NaN,-,-,NaN,-
4228,WP_238560621.1,-,"[(3.1.-.-, 0.1078561320900917), (2.3.1.48, 0.0...",-,NaN,-,-,NaN,-
4229,WP_238560622.1,-,"[(3.1.-.-, 0.227324977517128), (3.1.21.-, 0.00...",-,NaN,-,-,NaN,-
4230,WP_238560623.1,-,"[(3.1.-.-, 0.9987987279891968), (3.4.24.-, 0.0...",-,NaN,-,-,NaN,-


## 11. PRIAM

In [9]:
# # ! java -Xmx128G -jar /home/shizhenkun/codebase/DMLF/baselines/priam/PRIAM_search.jar -p /home/shizhenkun/codebase/DMLF/baselines/priam/PRIAM_JAN18 -i /home/shizhenkun/codebase/DMLF/data/shewanella.faa -o /home/shizhenkun/codebase/DMLF/results/case_shewanella/priam --blast_path /home/shizhenkun/downloads/blast-2.2.13/bin -np 100
res_priam = funclib.load_praim_res(f'{cfg.RESULTSDIR}/case_shewanella/priam/PRIAM_20230128102132/ANNOTATION/sequenceECs.txt')
test_res= test_res.merge(res_priam, on='id', how='left')
test_res.ec_priam = test_res.ec_priam.fillna('-')
test_res

,id,ec_dmlf,ec_dmlf_topk,ec_blast,pident_blast,ec_deepec,ec_ecpred,pident_ecpred,ec_catfam,ec_priam
0,WP_005054087.1,-,"[(3.1.-.-, 0.1445898711681366), (3.1.22.-, 0.0...",-,40.9,-,-,NaN,-,-
1,WP_006079870.1,-,"[(3.1.26.5, 0.11312833428382874), (3.1.1.29, 0...",-,100.0,3.1.26.5,-,NaN,-,-
2,WP_006079871.1,-,"[(2.7.7.6, 0.6336387395858765), (3.1.26.5, 0.0...",-,100.0,-,-,NaN,-,-
3,WP_006080725.1,-,"[(3.1.26.5, 0.10884372144937515), (2.7.7.6, 0....",-,100.0,-,-,NaN,-,-
4,WP_006080791.1,-,"[(3.1.-.-, 0.9882077574729919), (4.2.99.18, 0....",-,100.0,-,-,NaN,-,-
...,...,...,...,...,...,...,...,...,...,...
4227,WP_238560620.1,-,"[(3.1.-.-, 0.5166441202163696), (5.99.1.2, 0.0...",-,NaN,-,-,NaN,-,-
4228,WP_238560621.1,-,"[(3.1.-.-, 0.1078561320900917), (2.3.1.48, 0.0...",-,NaN,-,-,NaN,-,-
4229,WP_238560622.1,-,"[(3.1.-.-, 0.227324977517128), (3.1.21.-, 0.00...",-,NaN,-,-,NaN,-,-
4230,WP_238560623.1,-,"[(3.1.-.-, 0.9987987279891968), (3.4.24.-, 0.0...",-,NaN,-,-,NaN,-,-


## 12. Save 

In [10]:
def sup_ec_info(eclist):

    if eclist=='-':
        return ''
    df_ec_reaction = pd.read_csv(f'{cfg.DATADIR}explorenz/tb_ec_expasy_202211161459.csv')
    res_ec = []
    for item in eclist:
        res_ec = res_ec + item.split(',')
    res_ec = eclist.split(',')
    res_ec = list(set(res_ec))
    res_df = df_ec_reaction[df_ec_reaction.ec.isin(res_ec)]
    res_str = ''
    for index, row in res_df.iterrows():
        res_str = res_str + ','.join(row)
        res_str = res_str +' \n '
    return  res_str

test_res['ec_sup_info']=test_res.ec_dmlf.parallel_apply(lambda x: sup_ec_info(x))
test_res[test_res.ec_dmlf!='-']

,id,ec_dmlf,ec_dmlf_topk,ec_blast,pident_blast,ec_deepec,ec_ecpred,pident_ecpred,ec_catfam,ec_priam,ec_sup_info
31,WP_007647520.1,6.2.1.5,"[(6.2.1.5, 0.9972737431526184), (6.2.1.4, 0.06...",6.2.1.5,86.2,6.2.1.5,-,NaN,-,-,"6.2.1.5,Succinate--CoA ligase (ADP-forming).,A..."
38,WP_007651394.1,2.7.7.6,"[(2.7.7.6, 0.9996073842048645), (6.3.5.-, 0.00...",2.7.7.6,100.0,2.7.7.6,-,NaN,2.7.7.6,-,"2.7.7.6,DNA-directed RNA polymerase.,Nucleosid..."
41,WP_011070419.1,3.6.-.-,"[(3.6.-.-, 0.99997878074646), (3.6.1.-, 0.0002...",3.6.-.-,100.0,-,-,NaN,-,-,
44,WP_011070422.1,3.1.26.5,"[(3.1.26.5, 0.9996575117111206), (2.7.7.61, 0....",3.1.26.5,100.0,3.1.26.5,-,NaN,3.1.26.5,-,"3.1.26.5,Ribonuclease P.,Endonucleolytic cleav..."
49,WP_011070427.1,5.99.1.3,"[(5.99.1.3, 0.9993119239807129), (2.7.13.3, 0....",5.99.1.3,72.9,5.99.1.3,-,NaN,5.99.1.3,-,"5.99.1.3,Transferred entry: 5.6.2.2.,NONE \n"
...,...,...,...,...,...,...,...,...,...,...,...
4217,WP_238560609.1,"2.8.1.7,4.4.1.16","[(2.8.1.7, 0.9984127283096313), (4.4.1.16, 0.9...","4.4.1.16,2.8.1.7",50.3,2.8.1.7,-,NaN,2.8.1.7,-,"2.8.1.7,Cysteine desulfurase.,L-cysteine + acc..."
4219,WP_238560611.1,1.3.1.91,"[(1.3.1.91, 0.9939940571784973), (2.8.1.6, 0.0...",1.3.1.91,100.0,1.3.1.91,-,NaN,-,-,"1.3.1.91,tRNA-dihydrouridine(20) synthase (NAD..."
4222,WP_238560614.1,2.5.1.29,"[(2.5.1.29, 0.005744919180870056), (3.4.24.-, ...",-,NaN,-,-,NaN,-,-,"2.5.1.29,Geranylgeranyl diphosphate synthase.,..."
4223,WP_238560616.1,2.3.1.-,"[(2.3.1.-, 0.5374347567558289), (2.3.1.184, 0....",-,NaN,-,-,NaN,-,-,


In [11]:
#添加蛋白补充资料
sup_info = pd.read_csv(cfg.RESULTSDIR+'/case_shewanella/dmlf/proteins_1082_170432.csv')
test_res = test_res.merge(sup_info[['Protein product','Protein Name', 'Length']].rename(columns={'Protein product':'id'}), on='id', how='left')
test_res = test_res.drop_duplicates(subset=['id'], keep='first')
test_res.reset_index(drop=True, inplace=True)

In [24]:
test_res.to_excel(cfg.RESULTSDIR+'/case_shewanella/dmlf/case_shewanella.xlsx', index=None)

In [18]:
test_res

,id,ec_dmlf,ec_dmlf_topk,ec_blast,pident_blast,ec_deepec,ec_ecpred,pident_ecpred,ec_catfam,ec_priam,ec_sup_info,Protein Name,Length
0,WP_005054087.1,-,"[(3.1.-.-, 0.1445898711681366), (3.1.22.-, 0.0...",-,40.9,-,-,NaN,-,-,,MULTISPECIES: IS256-like element ISSod4 family...,400
1,WP_006079870.1,-,"[(3.1.26.5, 0.11312833428382874), (3.1.1.29, 0...",-,100.0,3.1.26.5,-,NaN,-,-,,MULTISPECIES: 50S ribosomal protein L28,78
2,WP_006079871.1,-,"[(2.7.7.6, 0.6336387395858765), (3.1.26.5, 0.0...",-,100.0,-,-,NaN,-,-,,MULTISPECIES: 50S ribosomal protein L33,57
3,WP_006080725.1,-,"[(3.1.26.5, 0.10884372144937515), (2.7.7.6, 0....",-,100.0,-,-,NaN,-,-,,MULTISPECIES: 30S ribosomal protein S21,71
4,WP_006080791.1,-,"[(3.1.-.-, 0.9882077574729919), (4.2.99.18, 0....",-,100.0,-,-,NaN,-,-,,MULTISPECIES: 50S ribosomal protein L19,117
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4197,WP_238560620.1,-,"[(3.1.-.-, 0.5166441202163696), (5.99.1.2, 0.0...",-,NaN,-,-,NaN,-,-,,DUF3560 domain-containing protein,373
4198,WP_238560621.1,-,"[(3.1.-.-, 0.1078561320900917), (2.3.1.48, 0.0...",-,NaN,-,-,NaN,-,-,,hypothetical protein,166
4199,WP_238560622.1,-,"[(3.1.-.-, 0.227324977517128), (3.1.21.-, 0.00...",-,NaN,-,-,NaN,-,-,,DUF4158 domain-containing protein,469
4200,WP_238560623.1,-,"[(3.1.-.-, 0.9987987279891968), (3.4.24.-, 0.0...",-,NaN,-,-,NaN,-,-,,hypothetical protein,74
